# Ollama + OpenAI + Python

## 1. ระบุชื่อโมเดล

หากคุณดึงโมเดลที่แตกต่างจาก "phi3:mini" มาใช้ ให้เปลี่ยนค่าที่เซลล์ด้านล่าง  
ตัวแปรนี้จะถูกใช้ในโค้ดตลอดทั้งโน้ตบุ๊ก


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. ตั้งค่า Open AI client

โดยทั่วไป OpenAI client จะถูกใช้งานร่วมกับ OpenAI.com หรือ Azure OpenAI เพื่อโต้ตอบกับโมเดลภาษาขนาดใหญ่  
อย่างไรก็ตาม มันสามารถใช้งานร่วมกับ Ollama ได้เช่นกัน เนื่องจาก Ollama มี endpoint ที่เข้ากันได้กับ OpenAI ที่ "http://localhost:11434/v1"


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. สร้างการตอบกลับในแชท

ตอนนี้เราสามารถใช้ OpenAI SDK เพื่อสร้างการตอบกลับสำหรับการสนทนาได้ คำขอนี้ควรสร้างไฮกุเกี่ยวกับแมว:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. การออกแบบคำสั่ง (Prompt Engineering)

ข้อความแรกที่ส่งไปยังโมเดลภาษาถูกเรียกว่า "ข้อความระบบ" หรือ "คำสั่งระบบ" ซึ่งมีหน้าที่กำหนดคำแนะนำโดยรวมสำหรับโมเดล  
คุณสามารถกำหนดคำสั่งระบบของคุณเองเพื่อแนะนำให้โมเดลภาษาแสดงผลลัพธ์ในรูปแบบที่แตกต่างออกไป  
ปรับแต่ง `SYSTEM_MESSAGE` ด้านล่างเพื่อให้โมเดลตอบเหมือนตัวละครในภาพยนตร์หรือรายการทีวีที่คุณชื่นชอบ หรือหาแรงบันดาลใจสำหรับคำสั่งระบบอื่นๆ ได้จาก [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts)

เมื่อคุณปรับแต่งข้อความระบบเรียบร้อยแล้ว ให้ส่งคำถามแรกของผู้ใช้ใน `USER_MESSAGE`


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. ตัวอย่างการใช้งานแบบ Few Shot

อีกวิธีหนึ่งในการแนะนำโมเดลภาษา คือการให้ "few shots" ซึ่งเป็นชุดตัวอย่างของคำถาม/คำตอบที่แสดงให้เห็นว่าโมเดลควรตอบสนองอย่างไร

ตัวอย่างด้านล่างนี้พยายามให้โมเดลภาษาแสดงบทบาทเป็นผู้ช่วยสอน โดยการให้ตัวอย่างคำถามและคำตอบที่ผู้ช่วยสอนอาจให้ และจากนั้นจึงให้โมเดลตอบคำถามที่นักเรียนอาจถาม

ลองใช้งานก่อน แล้วปรับ `SYSTEM_MESSAGE`, `EXAMPLES`, และ `USER_MESSAGE` ให้เหมาะสมกับสถานการณ์ใหม่


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. การสร้างเนื้อหาโดยใช้การดึงข้อมูลเสริม

RAG (Retrieval Augmented Generation) เป็นเทคนิคที่ช่วยให้โมเดลภาษาสามารถตอบคำถามได้อย่างแม่นยำในโดเมนเฉพาะ โดยเริ่มจากการดึงข้อมูลที่เกี่ยวข้องจากแหล่งความรู้ก่อน แล้วจึงสร้างคำตอบโดยอ้างอิงจากข้อมูลนั้น

เราได้เตรียมไฟล์ CSV ไว้ในเครื่องซึ่งมีข้อมูลเกี่ยวกับรถยนต์ไฮบริด โค้ดด้านล่างจะอ่านไฟล์ CSV ค้นหาข้อมูลที่ตรงกับคำถามของผู้ใช้ และสร้างคำตอบโดยอ้างอิงจากข้อมูลที่พบ โปรดทราบว่าสิ่งนี้จะใช้เวลานานกว่าตัวอย่างก่อนหน้า เนื่องจากต้องส่งข้อมูลจำนวนมากขึ้นไปยังโมเดล หากคุณสังเกตว่าคำตอบยังไม่อ้างอิงกับข้อมูล คุณสามารถลองปรับแต่งระบบหรือใช้โมเดลอื่น โดยทั่วไป RAG จะมีประสิทธิภาพมากขึ้นเมื่อใช้กับโมเดลขนาดใหญ่หรือเวอร์ชันที่ปรับแต่งของ SLMs


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้อง แต่โปรดทราบว่าการแปลโดยอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาดั้งเดิมควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษาจากผู้เชี่ยวชาญ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความผิดที่เกิดจากการใช้การแปลนี้
